In [1]:
from nltk.corpus import movie_reviews
import pandas
import numpy as np
import unicodedata
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

C:\Users\KONYAYEV.MAKSIM\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Скачка тулкита
#nltk.download()
#Позитивные и негативные ID
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [3]:
#Текст негативных отзывов
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

Task 1

Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [5]:
allfeats = posfeats + negfeats
allfeats = [list(x) for x in allfeats]
Y = [1] * len(posfeats) + [0] * len(negfeats)

IOError: [Errno 24] Too many open files: u'C:\\Users\\KONYAYEV.MAKSIM\\AppData\\Roaming\\nltk_data\\corpora\\movie_reviews\\pos\\cv000_29590.txt'

Task 2

Подсчитайте количество отзывов в выборке.

In [28]:
print "Количество отзывов в выборке ", len(posfeats) + len(negfeats)

 Количество отзывов в выборке  2000


Task 3

Подсчитайте долю класса 1 в выборке.

In [31]:
print "Доля класса 1 - ", len(posfeats) / float(len(allfeats))

Доля класса 1 -  0.5


Task 4

Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
neg_pos_Docks = [u" ".join(doc) for doc in allfeats]

In [82]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(neg_pos_Docks)
print "Количество признаков в новом пространстве = ",len(vectorizer.get_feature_names())
#vectorizer.transform([neg_pos_Docks[0]])

Количество признаков в новом пространстве =  39659


Task 5

Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [155]:
def text_classifier(vectorizer, transformer ,classifier):
    if (transformer is None) :
        return Pipeline(
                [("vectorizer", vectorizer),
                 ("classifier", classifier)]
            )
    else: 
        print "transformer"
        return Pipeline(
            [("vectorizer", vectorizer),
             ("transformer", transformer),
             ("classifier", classifier)]
        )

In [90]:
lg = LogisticRegression()
cross_val_score(text_classifier(CountVectorizer(), lg), neg_pos_Docks, Y, scoring = "accuracy").mean()

0.83602165039290777

Task 6

Аналогично accuracy, оцените качество по ROC AUC.

In [91]:
lg = LogisticRegression()
cross_val_score(text_classifier(CountVectorizer(), lg), neg_pos_Docks, Y, scoring = "roc_auc").mean()

0.91078250580140141

Task 7

Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [123]:
lg = LogisticRegression()
cv = CountVectorizer()
classifier = text_classifier(cv, lg).fit(neg_pos_Docks, Y)

In [131]:
abs_coef = [abs(w) for w in lg.coef_[0]]
indexes = sorted(range(len(abs_coef)), key=lambda i: abs_coef[i], reverse=True)[:5]
print [ cv.get_feature_names()[index] for index in indexes]
print [ lg.coef_[0][index] for index in indexes]

[u'bad', u'unfortunately', u'worst', u'fun', u'waste']
[-0.78225361709067653, -0.63662392955497649, -0.5928385327017055, 0.55605126142442174, -0.50819138655023299]


In [132]:
def write_answer(fileName, value):
    with open(fileName+".txt", "w") as fout:
        fout.write(str(value))

In [134]:
write_answer("task1", "2000")
write_answer("task2", "0.5")
write_answer("task3", "39659")
write_answer("task4", "0.83602165039290777")
write_answer("task5", "0.91078250580140141")
write_answer("task6", "bad unfortunately")


WEEK 2

Task 1 

Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

In [141]:
#а) pipeline из CountVectorizer() и LogisticRegression()
lg = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(), lg), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",results.mean(), "Среднеквадратичное отклонение - ", results.std()

 Среднее -  0.841 Среднеквадратичное отклонение -  0.0167779617356


In [176]:
#б) pipeline из TfidfVectorizer() и LogisticRegression()
from sklearn.feature_extraction.text import TfidfVectorizer
lg = LogisticRegression()
results = cross_val_score(text_classifier(TfidfVectorizer(), None,lg), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",results.mean(), "Среднеквадратичное отклонение - ", results.std()

Среднее -  0.821 Среднеквадратичное отклонение -  0.00406201920232


Task 2

Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

In [161]:
lg = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(min_df=10), None ,lg), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)
lg = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(min_df=50), None ,lg), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

Среднее -  0.839 Среднеквадратичное отклонение -  0.0118953772534
Среднее -  0.813 Среднеквадратичное отклонение -  0.0134536240471


Task 3

Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

In [165]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

classifier = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(), None ,classifier), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

classifier = LinearSVC()
results = cross_val_score(text_classifier(CountVectorizer(), None ,classifier), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

classifier = SGDClassifier()
results = cross_val_score(text_classifier(CountVectorizer(), None ,classifier), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

Среднее -  0.841 Среднеквадратичное отклонение -  0.0167779617356
Среднее -  0.8325 Среднеквадратичное отклонение -  0.0162788205961


C:\Users\KONYAYEV.MAKSIM\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Среднее -  0.7825 Среднеквадратичное отклонение -  0.042514703339


Task 4

Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

In [168]:
import nltk
nl_stopWords = nltk.corpus.stopwords.words('english')

In [169]:
classifier = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(stop_words=nl_stopWords), None ,classifier), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

Среднее -  0.841 Среднеквадратичное отклонение -  0.0112472218792


In [170]:
classifier = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(stop_words="english"), None ,classifier), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

Среднее -  0.839 Среднеквадратичное отклонение -  0.00982344135219


Task 5

Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [171]:
classifier = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(ngram_range=(1,2)), None ,classifier), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

Среднее -  0.8525 Среднеквадратичное отклонение -  0.0165075740192


In [174]:
classifier = LogisticRegression()
results = cross_val_score(text_classifier(CountVectorizer(ngram_range=(3,5), analyzer = "char_wb" ), None ,classifier), neg_pos_Docks, Y, scoring = "accuracy", cv=5)
print "Среднее - ",np.mean(results), "Среднеквадратичное отклонение - ", np.std(results)

Среднее -  0.8205 Среднеквадратичное отклонение -  0.0100498756211


In [175]:
write_answer("task1_w2", "0.841 0.0167779617356 0.821 0.00406201920232")
write_answer("task2_w2", "0.839 0.813")
write_answer("task3_w2", "0.7825")
write_answer("task4_w2", "0.841 0.839")
write_answer("task5_w2", "0.8525 0.8205")